# Notebook de récupération des données

Doit être exécuté en premier.

In [1]:
import requests
import zipfile
import io
import pandas as pd
from shutil import rmtree
from jyquickhelper import add_notebook_menu
from unidecode import unidecode

add_notebook_menu()

## 1 - Acquisition des données

### Récupération des données socio-économiques 

Disponibles sur le site de l'INSEE.

In [2]:
url = 'https://www.insee.fr/fr/statistiques/fichier/5359146/dossier_complet.zip'

In [3]:
# Attention, l'éxécution de cette cellule peut prendre un certain temps
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("INSEE/")

### Récupération des données des élections municipales de 2020

Disponibles sur le site data.gouv.

In [4]:
municipales_2020_T1 = pd.read_excel('https://static.data.gouv.fr/resources/elections-municipales-2020-liste-des-candidats-elus-au-t1-et-liste-des-communes-entierement-pourvues/20200617-160846/maires-au-17-06-2020-vf.xlsx')
municipales_2020_T2 = pd.read_excel('https://static.data.gouv.fr/resources/municipales-2020-resultats-2nd-tour/20200629-192435/2020-06-29-resultats-t2-communes-de-1000-hab-et-plus.xlsx')

### Récupération des données des élections municipales de 2014

Il s'agit de la seule élection où une liste finale des maires élus existe.

In [5]:
municipales_2014 = pd.read_excel('https://www.data.gouv.fr/storage/f/2014-06-17T12-56-58/maires-17-06-2014.xlsx')

## 2 - Nettoyage des données et présentation

### Elections municipales de 2020

In [6]:
# Renommer les colonnes du premier tour (attention code non idem-potent)
municipales_2020_T1.columns = municipales_2020_T1.iloc[0]
municipales_2020_T1.drop(labels=[0], axis=0, inplace=True)
municipales_2020_T1.reset_index(drop=True, inplace=True)

# On renomme la colonne du nom de l'élu
municipales_2020_T1.rename(columns={"Nom de l'élu": "Nom de l'élu en 2020"}, inplace=True)

In [7]:
# Ajout du code postal (en réalité code commune, mais renommé ainsi pour éviter la confusion avec 'Code de la commune')

# Ajout d'un 0 si besoin devant le numéro du département
def change_dep(dep):
    try:
        if int(dep)<10:
            return "0"+str(dep)
        return dep
    except:
        return dep

# Ajout des 0 devant le numéro du département et transition int vers string
def change_commune(com):
    try:
        if com <10:
            return "00"+str(com)
        elif com<100:
            return "0" + str(com)
        else :
            return str(com)
    except:
        return str(com)

municipales_2020_T1['Code du département (Maire)'] = municipales_2020_T1['Code du département (Maire)'].apply(lambda ligne: change_dep(ligne))
municipales_2020_T1['Code Insee de la commune'] = municipales_2020_T1['Code Insee de la commune'].apply(lambda ligne: change_commune(ligne))

# Cration de la colonne avec le code postal
municipales_2020_T1['Code postal'] = municipales_2020_T1['Code du département (Maire)'].astype(str) + municipales_2020_T1['Code Insee de la commune'].astype(str)

In [8]:
municipales_2020_T1

,Code du département (Maire),Libellé de département (Maires),Code Insee de la commune,Libellé de la commune,Nom de l'élu en 2020,Prénom de l'élu,Code sexe,Date de naissance,Code profession,Libellé de la profession,Date de début de la fonction,Code postal
0,01,AIN,001,L'Abergement-Clémenciat,BOULON,Daniel,M,1951-03-04 00:00:00,74,Anciens cadres,2020-05-26 00:00:00,01001
1,01,AIN,002,L'Abergement-de-Varey,ORSET,Max,M,1947-11-02 00:00:00,65,"Ouvriers qualifiés de la manutention, du magas...",2020-05-27 00:00:00,01002
2,01,AIN,004,Ambérieu-en-Bugey,FABRE,Daniel,M,1961-09-07 00:00:00,74,Anciens cadres,2020-05-28 00:00:00,01004
3,01,AIN,005,Ambérieux-en-Dombes,PERNET,Pierre,M,1961-07-29 00:00:00,34,"Professeurs, professions scientifiques",2020-05-27 00:00:00,01005
4,01,AIN,006,Ambléon,BIONDA,Annie,F,1951-11-28 00:00:00,77,Anciens employés,2020-05-25 00:00:00,01006
...,...,...,...,...,...,...,...,...,...,...,...,...
28382,ZP,POLYNESIE FRANCAISE,715,Faa a,TEMARU,"Oscar, Manutahi",M,1944-11-01 00:00:00,74,Anciens cadres,2020-05-23 00:00:00,ZP715
28383,ZP,POLYNESIE FRANCAISE,737,Puka Puka,VILLANT,Raphaël,M,1961-02-08 00:00:00,54,Employés administratifs d'entreprise,2020-05-25 00:00:00,ZP737
28384,ZP,POLYNESIE FRANCAISE,738,Punaauia,LISSANT,Simplicio,M,1962-08-02 00:00:00,74,Anciens cadres,2020-05-23 00:00:00,ZP738
28385,ZP,POLYNESIE FRANCAISE,741,Rapa,NARII,Tuanainai,M,1963-08-27 00:00:00,37,Cadres administratifs et commerciaux d'entreprise,2020-05-25 00:00:00,ZP741


In [9]:
# Renommer les colonnes du second tour
nom_cols_T2 = list(municipales_2020_T2.columns)

for i in range(30,78):
    nom_cols_T2[i] = nom_cols_T2[i-12]
    
for i in range(30,78):
    nom_cols_T2[i] = nom_cols_T2[i] + "." + str(int((i-18)/12))
    
municipales_2020_T2.columns = nom_cols_T2

In [10]:
# Ajout du code postal (en réalité code commune, mais renommé ainsi pour éviter la confusion avec 'Code de la commune')
municipales_2020_T2['Code postal'] = municipales_2020_T2['Code du département'] + municipales_2020_T2['Code de la commune']

In [11]:
municipales_2020_T2

,Code du département,Libellé du département,Code de la commune,Libellé de la commune,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,...,Nom.4,Prénom.4,Liste.4,Sièges / Elu.4,Sièges Secteur.4,Sièges CC.4,Voix.4,% Voix/Ins.4,% Voix/Exp.4,Code postal
0,01,Ain,034,Belley,5757,2877,49.97,2880,50.03,51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01034
1,01,Ain,160,Ferney-Voltaire,3798,2280,60.03,1518,39.97,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01160
2,01,Ain,194,Jassans-Riottier,4357,2689,61.72,1668,38.28,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01194
3,01,Ain,249,Miribel,6847,4064,59.35,2783,40.65,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01249
4,01,Ain,354,Saint-Genis-Pouilly,5015,3108,61.97,1907,38.03,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1424,ZP,Polynésie française,728,Maupiti,1085,54,4.98,1031,95.02,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZP728
1425,ZP,Polynésie française,733,Paea,9180,2771,30.19,6409,69.81,40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZP733
1426,ZP,Polynésie française,734,Papara,8597,3290,38.27,5307,61.73,27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZP734
1427,ZP,Polynésie française,735,Papeete,18056,7989,44.25,10067,55.75,74,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZP735


### Elections municipales de 2014

In [12]:
# Renommer les colonnes (attention code non idem-potent)
municipales_2014.columns = municipales_2014.iloc[1]
municipales_2014.drop(labels=[0,1], axis=0, inplace=True)
municipales_2014.reset_index(drop=True, inplace=True)

# On renomme la colonne du nom de l'élu
municipales_2014.rename(columns={"Nom de l'élu": "Nom de l'élu en 2014"}, inplace=True)

In [13]:
# Travail préparatoire nécessaire à la création du code postal
municipales_2014['Code du département (Maire)'] = municipales_2014['Code du département (Maire)'].apply(lambda ligne: change_dep(ligne))
municipales_2014['Code Insee de la commune'] = municipales_2014['Code Insee de la commune'].apply(lambda ligne: change_commune(ligne))

# Cration de la colonne avec le code postal
municipales_2014['Code postal'] = municipales_2014['Code du département (Maire)'].astype(str) + municipales_2014['Code Insee de la commune'].astype(str)

In [14]:
municipales_2014

1,Code du département (Maire),Libellé de département (Maires),Code Insee de la commune,Libellé de la commune,Population de la commune,Nom de l'élu en 2014,Prénom de l'élu,Code sexe,Date de naissance,Code profession,Libellé de la profession,Code postal
0,01,AIN,001,L'Abergement-Clémenciat,780,BOULON,Daniel,M,1951-03-04 00:00:00,61,Retraités salariés privés,01001
1,01,AIN,002,L'Abergement-de-Varey,234,ORSET,Max,M,1947-11-02 00:00:00,65,Autres retraités,01002
2,01,AIN,004,Ambérieu-en-Bugey,13839,FABRE,Daniel,M,1961-09-07 00:00:00,51,Cadres (entreprises publiques),01004
3,01,AIN,005,Ambérieux-en-Dombes,1600,PERNET,Pierre,M,1961-07-29 00:00:00,42,Professeurs du secondaire et techn.,01005
4,01,AIN,006,Ambléon,112,BIONDA,Annie,F,1951-11-28 00:00:00,63,Retraités fonct.publique (sf enseig.),01006
...,...,...,...,...,...,...,...,...,...,...,...,...
36623,ZP,POLYNESIE FRANCAISE,056,Ua-Huka,621,OHU,Nestor,M,1962-04-02 00:00:00,52,Employés (autres entrep. publiques),ZP056
36624,ZP,POLYNESIE FRANCAISE,057,Ua-Pou,2173,KAIHA,Joseph,M,1960-12-07 00:00:00,43,Enseignants 1er deg.-directeurs école,ZP057
36625,ZP,POLYNESIE FRANCAISE,058,Uturoa,3697,TEROOATEA,Sylviane,F,1966-10-13 00:00:00,50,Cadres sup. (entreprises publiques),ZP058
36626,ZS,SAINT PIERRE ET MIQUELON,501,Miquelon-Langlade,635,DE LIZARRAGA,Jean,M,1943-12-17 00:00:00,63,Retraités fonct.publique (sf enseig.),ZS501


## 3 - Détermination des maires de 2020

### Second tour des élections municipales de 2020

Le base de données du premier tour tour fournit directement les maires élus. Cependant, dans celle du second tour, il n'y a que les scores de chaque candidat. Il faut donc déterminer les maires élus au decond tour.

In [15]:
# Combien y a-t-il de candidats au second tour ?
def compte_candidats(liste):
    return len(liste) - sum(pd.isna(liste).astype(int))

In [16]:
# Création de listes qui serviront à déterminer les maires élus
communes = municipales_2020_T2['Code postal'].unique()

noms = [f'Nom.{i}' for i in range(1,5)]
noms.insert(0, 'Nom')

voix = [f'Voix.{i}' for i in range(1,5)]
voix.insert(0, 'Voix')

muni2 = municipales_2020_T2.copy(deep=True)

In [17]:
# Détermination des maires élus au second tour
maires = pd.DataFrame(columns=['Code postal', "Nom de l'élu en 2020"])

for i in communes:
    # On parcourt les villes qui ont connu un second tour
    nombre_candidats = compte_candidats(list(muni2.loc[muni2['Code postal']==i][noms].iloc[0]))   
    max_voix = 0
    indice_max = 0
    
    # On regarde quel candidat obtient le plus de voix, qui sera le maire élu
    if nombre_candidats == 1:
        maires.loc[len(maires)] = [i, muni2.loc[muni2['Code postal']==i]['Nom'].unique()]
    else:    
        for j in range(nombre_candidats):
            actuel = muni2.loc[muni2['Code postal']==i][voix[j]].sum()
            if actuel > max_voix:
                max_voix = actuel
                indice_max = j
        maires.loc[len(maires)] = [i, muni2.loc[muni2['Code postal']==i][noms[indice_max]].unique()]

On observe une singularité pour les communes de moins de 1 000 habitants. Pour ces communes, l'élection des conseillers municipaux se déroule au scrutin majoritaire plurinominal à deux tours, avec panachage. La somme des voix de tous les candidats peut donc être supérieure au nombre de votants.

Nous allons donc ne pas pouvoir analyser ces communes.

In [18]:
# Rendons les noms des maires plus lisibles en les mettant au format string
def normalise_nom(nom):
    return str(nom)[2:-2]

maires["Nom de l'élu en 2020"] = maires["Nom de l'élu en 2020"].apply(lambda ligne: normalise_nom(ligne))

In [19]:
maires

,Code postal,Nom de l'élu en 2020
0,01034,LAHUERTA
1,01160,RAPHOZ
2,01194,REVERCHON
3,01249,GAITET
4,01354,BERTRAND
...,...,...
1424,ZP728,RAUFAUORE
1425,ZP733,GEROS
1426,ZP734,PUNUA
1427,ZP735,BUILLARD


### Agrégation des deux tours pour obtenir tous maires élus en 2020

In [20]:
municipales_2020 = pd.concat([municipales_2020_T1[['Code postal', "Nom de l'élu en 2020"]], maires], axis=0)

In [21]:
# Ensemble des maires élus en 2020
municipales_2020

,Code postal,Nom de l'élu en 2020
0,01001,BOULON
1,01002,ORSET
2,01004,FABRE
3,01005,PERNET
4,01006,BIONDA
...,...,...
1424,ZP728,RAUFAUORE
1425,ZP733,GEROS
1426,ZP734,PUNUA
1427,ZP735,BUILLARD


### Agrégation des résultats des années 2014 et 2020

Faisons l'aggrégation des maires élus en 2014 et 2020 par ville.
Attention : il existe des données manquantes car son également incluses certains villes de moins de 1000 habitants. Cela sera mis à jour plus tard avec la dernière base de données.

In [22]:
municipales_2014_et_2020 = municipales_2020.copy(deep= True)
municipales_2014_et_2020 = municipales_2014_et_2020.merge(municipales_2014[["Code postal", "Nom de l'élu en 2014", 'Libellé de la commune']], on = 'Code postal')

In [23]:
municipales_2014_et_2020

,Code postal,Nom de l'élu en 2020,Nom de l'élu en 2014,Libellé de la commune
0,01001,BOULON,BOULON,L'Abergement-Clémenciat
1,01002,ORSET,ORSET,L'Abergement-de-Varey
2,01004,FABRE,FABRE,Ambérieu-en-Bugey
3,01005,PERNET,PERNET,Ambérieux-en-Dombes
4,01006,BIONDA,BIONDA,Ambléon
...,...,...,...,...
29641,ZN824,DEDANE,DALAP,Pouébo
29642,ZN826,TIDJINE EP HMAE,TIDJINE EP HMAE,Poum
29643,ZN827,GORO ATU,KURTOVITCH EP METZDORF,Poya
29644,ZN831,BOATATE KOLEKOLE,DOUNEHOTE,Voh


## 4 - Maires candidats à leurs successions 

Déterminons si les maires de 2014 ont tenté de se faire réélire en 2020. Pour cela on utilise la base de données de data.gouv qui comprend tous les candidats aux élections municipales de 2020.

In [24]:
# Attention, l'éxécution de cette cellule peut prendre un certain temps
candidats = pd.read_excel('https://www.data.gouv.fr/fr/datasets/r/1f67d380-6810-4f93-a952-80ee18c50d78')

In [25]:
noms = [f'Nom.{i}' for i in range(1,62)]
noms.insert(0, 'Nom')

In [26]:
# Fonction qui va servir à donner le même format à tous les noms
def mise_aux_normes(ligne):
    try:
        return unidecode(ligne).replace('-', ' ')
    except:
        return ligne

In [27]:
# Mise aux normes de certaines colonnes, parce que dans les différentes bases de données, 
# les noms ne respectent pas exactement les mêmes standards.
candidats['Code du département'] = candidats['Code du département'].apply(lambda ligne: change_dep(ligne))
candidats['Code de la commune'] = candidats['Code de la commune'].apply(lambda ligne: change_commune(ligne))
candidats[noms] = candidats[noms].apply(lambda ligne: mise_aux_normes(ligne))
municipales_2014_et_2020['Nom de l\'élu en 2020'] = municipales_2014_et_2020['Nom de l\'élu en 2020'].apply(lambda ligne: mise_aux_normes(ligne))
municipales_2014_et_2020['Nom de l\'élu en 2014'] = municipales_2014_et_2020['Nom de l\'élu en 2014'].apply(lambda ligne: mise_aux_normes(ligne))

# Création de la colonne avec le code postal
candidats['Code postal'] = candidats['Code du département'].astype(str) + candidats['Code de la commune'].astype(str)

In [28]:
# Attention, l'éxécution de cette cellule peut prendre un certain temps

municipales_2014_et_2020['Noms différents'] = municipales_2014_et_2020.apply(lambda row: row["Nom de l'élu en 2014"] != row["Nom de l'élu en 2020"], axis=1)
municipales_2014_et_2020['Tentative de réélection'] = True
df =  municipales_2014_et_2020.loc[municipales_2014_et_2020['Noms différents']]
df2 =  municipales_2014_et_2020.loc[municipales_2014_et_2020['Noms différents']==False]

for cpt, maire in enumerate(municipales_2014_et_2020.loc[municipales_2014_et_2020['Noms différents']]["Nom de l'élu en 2014"]):
    # Si le maire de 2020 n'est pas celui de 2014, on regarde s'il a tenté de se faire réélire
    ville = municipales_2014_et_2020.loc[municipales_2014_et_2020['Noms différents']]['Code postal'].iloc[cpt]
    df.iloc[cpt, df.columns.get_loc('Tentative de réélection')] = maire in list(candidats.loc[candidats['Code postal']==ville][noms].iloc[0])

In [29]:
municipales_2014_et_2020 = pd.concat([df, df2])
municipales_2014_et_2020.drop_duplicates(subset=['Code postal'], inplace=True)
municipales_2014_et_2020

,Code postal,Nom de l'élu en 2020,Nom de l'élu en 2014,Libellé de la commune,Noms différents,Tentative de réélection
5,01007,BARILLOT,LEVRAT,Ambronay,True,False
7,01009,GUILLERMIN,DELARUELLE,Andert-et-Condon,True,False
9,01011,DOCHE,GIRARD,Apremont,True,False
11,01014,CRACCHIOLO,MAISSIAT,Arbent,True,False
12,01015,RIERA,BERGER,Arbignieu,True,False
...,...,...,...,...,...,...
29629,ZN805,NATUREL,NATUREL,Dumbéa,False,True
29634,ZN812,WEISS,WEISS,Koumac,False,True
29640,ZN822,NEAOUTYINE,NEAOUTYINE,Poindimié,False,True
29642,ZN826,TIDJINE EP HMAE,TIDJINE EP HMAE,Poum,False,True


## 5 - Fusion avec les données de l'INSEE

### Création d'un label commun: CODGEO

L'INSEE utilise un "Code géographique" (colonne CODGEO) pour identifier les communes, que nous n'avons pas pour les communes dont le maire a été élu au premier tour.
Afin de fusionner les bases de données socio-économiques et des élections, nous avons donc besoin de récupérer ce code géographique.

In [30]:
# Meta-données de la base de données de l'INSEE
meta_insee = pd.read_csv(filepath_or_buffer='INSEE/meta_dossier_complet.csv', sep=';', encoding='l5')

# Données de la base de données de l'INSEE
insee = pd.read_csv(filepath_or_buffer='INSEE/dossier_complet.csv', sep=';', encoding='l5')

/var/folders/99/gq6w32xj1_b7n4b18rr2czgr0000gn/T/ipykernel_4958/2173571963.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  insee = pd.read_csv(filepath_or_buffer='INSEE/dossier_complet.csv', sep=';', encoding='l5')


In [31]:
# Permet de garder uniquement les CODGEO, car la colonne contient soit le code CODGEO, soit un NA si c'était un autre type de variable
meta_insee = meta_insee[['COD_MOD', 'LIB_MOD']].dropna()
meta_insee.reset_index(drop=True, inplace=True)

In [32]:
# Pour pallier les problèmes d'encodage
def problème_FR(phrase):
    return phrase.replace('Ã©', 'é').replace('Ã«', 'ë').replace('Ã¨', 'è').replace('Ã\x89', 'É').replace('Ã´', 'ô').replace('Ã§', 'ç').replace('Ã®', 'î').replace('Ã\x8b', 'Ë').replace('Ã\x88', 'È').replace('Ã¯', 'ï').replace('Ã¢','â').replace('Ãª', 'ê').replace('Å\x93', 'œ').replace('Ã¿', 'ÿ').replace('Å\x92', 'Œ').replace('Ã¼', 'ü').replace('Ã»', 'û')

In [33]:
meta_insee['LIB_MOD'] = meta_insee['LIB_MOD'].apply(lambda ligne: problème_FR(ligne))
meta_insee.columns = ['CODGEO', 'Libellé de la commune']

In [34]:
meta_insee

,CODGEO,Libellé de la commune
0,01001,L'Abergement-Clémenciat
1,01002,L'Abergement-de-Varey
2,01004,Ambérieu-en-Bugey
3,01005,Ambérieux-en-Dombes
4,01006,Ambléon
...,...,...
34995,75116,Paris 16e Arrondissement
34996,75117,Paris 17e Arrondissement
34997,75118,Paris 18e Arrondissement
34998,75119,Paris 19e Arrondissement


### Gestion des homonymes

In [35]:
# Le nom Saint-Colombe est le plus porté dans notre base de données, avec 11 communes portant ce nom
sainte_colombe = municipales_2014_et_2020.loc[municipales_2014_et_2020['Libellé de la commune']=='Sainte-Colombe']
sainte_colombe

,Code postal,Nom de l'élu en 2020,Nom de l'élu en 2014,Libellé de la commune,Noms différents,Tentative de réélection
1556,05135,ROUX,ALMERAS,Sainte-Colombe,True,True
4850,17319,PERUFFO,POURTEAU,Sainte-Colombe,True,True
10957,35262,MALNOE,PETIBON,Sainte-Colombe,True,False
13975,46260,LANDES,LABARTHE,Sainte-Colombe,True,True
22046,69189,DELEIGUE,MASSE,Sainte-Colombe,True,False
24175,76569,BUREL,COLOMBEL,Sainte-Colombe,True,False
5804,21544,FAURE STERNAD,FAURE STERNAD,Sainte-Colombe,False,True
7200,25515,MALFROY,MALFROY,Sainte-Colombe,False,True
10377,33390,THIBEAU,THIBEAU,Sainte-Colombe,False,True
12496,40252,DUTOYA,DUTOYA,Sainte-Colombe,False,True


Certaines communes possédant des homonymes, nous devons ajouter une autre façon de discerner less communes. En effet, un même nom peut être porté par différentes villes, mais jamais dans le même département.

In [36]:
municipales_2014_et_2020['Code du département'] = municipales_2014_et_2020['Code postal'].apply(lambda row: row[:2])
meta_insee['Code du département'] = meta_insee['CODGEO'].apply(lambda row: row[:2])

En récupérant le Code du département, nous remarquons une différence de traitement pour les communes d'outre-mer selon les bases.

In [37]:
meta_insee['Code du département'].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '2A', '2B', '30', '31', '32',
       '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43',
       '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54',
       '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65',
       '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76',
       '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87',
       '88', '89', '90', '91', '92', '93', '94', '95', '97'], dtype=object)

In [38]:
municipales_2014_et_2020['Code du département'].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '2A', '2B', '30', '31', '32',
       '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43',
       '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54',
       '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65',
       '66', '67', '68', '69', '70', '71', '72', '73', '74', '76', '77',
       '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88',
       '89', '90', '91', '92', '93', '94', '95', 'ZA', 'ZB', 'ZC', 'ZN',
       'ZD'], dtype=object)

In [39]:
municipales_2014_et_2020.loc[municipales_2014_et_2020['Libellé de la commune']=='Kourou']

,Code postal,Nom de l'élu en 2020,Nom de l'élu en 2014,Libellé de la commune,Noms différents,Tentative de réélection,Code du département
28261,ZC304,RINGUET,RINGUET,Kourou,False,True,ZC


In [40]:
meta_insee.loc[meta_insee['Libellé de la commune']=='Kourou']

,CODGEO,Libellé de la commune,Code du département
34895,97304,Kourou,97


Afin de permettre la correspondance entre les bases, on remplace 'ZA', 'ZB', 'ZC', 'ZN' et 'ZD' par '97'.

In [41]:
def Z_97(row):
    return row.replace('ZA', '97').replace('ZB', '97').replace('ZC', '97').replace('ZN', '97').replace('ZD', '97')

In [42]:
municipales_2014_et_2020['Code du département'] = municipales_2014_et_2020['Code du département'].apply(lambda row: Z_97(row))

### Fusion des bases de données

In [43]:
# Fusion de municipales_2014_et_2020 et meta_insee, pour que chaque commune puisse avoir son CODGEO
municipales_2014_et_2020 = municipales_2014_et_2020.merge(meta_insee, on = ['Libellé de la commune', 'Code du département'], how='inner')

In [44]:
municipales_2014_et_2020.drop(columns=['Code du département', 'Noms différents'], inplace=True)
municipales_2014_et_2020.reset_index(drop=True, inplace=True)
municipales_2014_et_2020

,Code postal,Nom de l'élu en 2020,Nom de l'élu en 2014,Libellé de la commune,Tentative de réélection,CODGEO
0,01007,BARILLOT,LEVRAT,Ambronay,False,01007
1,01009,GUILLERMIN,DELARUELLE,Andert-et-Condon,False,01009
2,01011,DOCHE,GIRARD,Apremont,False,01011
3,01014,CRACCHIOLO,MAISSIAT,Arbent,False,01014
4,01022,DESCHAMPS,CHARMONT MUNET,Artemare,True,01022
...,...,...,...,...,...,...
28144,ZD408,MIRANVILLE,MIRANVILLE,La Possession,True,97408
28145,ZD420,GIRONCEL,GIRONCEL,Sainte-Suzanne,True,97420
28146,ZD422,THIEN AH KOON,THIEN AH KOON,Le Tampon,True,97422
28147,ZD423,PAUSE,PAUSE,Les Trois-Bassins,True,97423


In [45]:
# Pour la standardisation
def to_int(ligne):
    try:
        return int(ligne)
    except:
        return ligne

In [46]:
municipales_2014_et_2020['CODGEO'] = municipales_2014_et_2020['CODGEO'].apply(lambda ligne: to_int(ligne))

Nous pouvons maintenant ajouter aux données de l'INSEE, les données relatives aux élections.

In [47]:
# Fusion de municipales_2014_et_2020 et insee, grâce à la colonne commune CODGEO
insee = insee.merge(municipales_2014_et_2020, on='CODGEO')

In [48]:
insee

,CODGEO,P19_POP,P19_POP0014,P19_POP1529,P19_POP3044,P19_POP4559,P19_POP6074,P19_POP7589,P19_POP90P,P19_POPH,...,RTUH22,RTLIT22,AJCS22,AJCSUH22,AJCSLIT22,Code postal,Nom de l'élu en 2020,Nom de l'élu en 2014,Libellé de la commune,Tentative de réélection
0,1001,779.0,158.508629,102.468339,131.523972,194.112147,131.050309,56.014471,5.322133,397.885214,...,0.0,0.0,0.0,0.0,0.0,01001,BOULON,BOULON,L'Abergement-Clémenciat,True
1,1002,256.0,58.646351,29.609476,74.028483,38.684019,35.381928,17.582020,2.067723,129.517516,...,0.0,0.0,0.0,0.0,0.0,01002,ORSET,ORSET,L'Abergement-de-Varey,True
2,1004,14134.0,2642.167045,3076.783602,2730.398468,2391.347109,2054.139468,1044.007592,195.156716,6790.643609,...,16.0,16.0,0.0,0.0,0.0,01004,FABRE,FABRE,Ambérieu-en-Bugey,True
3,1005,1751.0,346.859147,268.250842,362.580808,380.267677,272.181257,111.034231,9.826038,877.465208,...,0.0,0.0,0.0,0.0,0.0,01005,PERNET,PERNET,Ambérieux-en-Dombes,True
4,1006,112.0,12.218182,16.290909,15.272727,29.527273,27.490909,10.181818,1.018182,63.127273,...,0.0,0.0,0.0,0.0,0.0,01006,BIONDA,BIONDA,Ambléon,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27595,97420,24065.0,5706.444105,4877.987702,4640.069455,5256.374356,2604.670639,902.029424,77.424318,11489.291675,...,0.0,0.0,0.0,0.0,0.0,ZD420,GIRONCEL,GIRONCEL,Sainte-Suzanne,True
27596,97421,7136.0,1711.000000,1321.000000,1289.000000,1575.000000,858.000000,354.000000,28.000000,3655.000000,...,0.0,0.0,0.0,0.0,0.0,ZD421,FOUASSIN,FOUASSIN,Salazie,True
27597,97422,79824.0,17965.710576,16081.862311,14636.246671,16582.101722,10535.209215,3678.636613,344.232891,38326.445210,...,0.0,0.0,0.0,0.0,0.0,ZD422,THIEN AH KOON,THIEN AH KOON,Le Tampon,True
27598,97423,7015.0,1389.693054,1286.152394,1284.135423,1708.032693,947.626169,349.944477,49.415791,3473.207273,...,0.0,0.0,0.0,0.0,0.0,ZD423,PAUSE,PAUSE,Les Trois-Bassins,True


### Export

In [49]:
insee.to_csv('Données élections et INSEE.csv', index = False)

In [50]:
# Suppression du dossier qui contenait les données de l'INSEE
rmtree('INSEE/', ignore_errors=False, onerror=None)

## 6 - Origine des pertes

On observe une perte entre le nombre de communes recensées par l'INSEE, environ 35 000, et le nombre final de communes, 27600. On recense deux principaux facteurs.

### Qualité des données

Le principal facteur en terme d'impact est la qualité des bases de données que nous avons pu récupérer:

In [51]:
print('Nombre de communes disponibles pour 2014:', municipales_2014.shape[0])
print('Nombre de communes disponibles pour 2020:', municipales_2020_T1.shape[0] + municipales_2020_T2.shape[0])
print('Soit une perte de', municipales_2014.shape[0] - municipales_2020_T1.shape[0] - municipales_2020_T2.shape[0], 'communes.')

Nombre de communes disponibles pour 2014: 36628
Nombre de communes disponibles pour 2020: 29816
Soit une perte de 6812 communes.


Il s'agit pour l'essentiel de petites communes, mais également de grandes villes. Par exemple, Paris et Marseille ne sont pas disponibles dans nos bases de 2020. La perte des petites villes était anticipée car notre base du second tour ne contenait que les villes de plus de 1000 habitants, mais la perte de villes plus importantes n'est pas expliquée.

In [52]:
municipales_2020_T2.loc[municipales_2020_T2['Libellé de la commune']=='Paris']

,Code du département,Libellé du département,Code de la commune,Libellé de la commune,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,...,Nom.4,Prénom.4,Liste.4,Sièges / Elu.4,Sièges Secteur.4,Sièges CC.4,Voix.4,% Voix/Ins.4,% Voix/Exp.4,Code postal


### Dénomination différentes des communes

Avant la partie 5 sur la fusions des données, nos données recensent encore 29642 communes. Déjà, une petite partie des communes ne possédait pas la même dénomination dans les bases de 2014 et 2020, ce qui entraînait une petite perte de l'ordre de 150 communes.

Cependant il existe une important différence dans les noms des communes selon l'INSEE et selon data.gouv. Certaines communes ont changé de nom en cours de temps, d'autres ont des dénominations spéciales, d'autres ont des découpages électoraux particuliers, d'autres possèdent des tirets dans une base mais pas dans une autre...

Il n'est pas possible de lister ici toutes les différences, minimes, qui existent entre ces bases. Mais mises bout à bout, elles entaînent la perte de l'ordre de 2000 communes supplémentaires, aboutissant à retenir uniquement 27 600 communes.

In [53]:
# Exemple de différence de nom:
meta_insee.loc[meta_insee['Libellé de la commune']=='Sainte-Colombe-en-Auxois']

,CODGEO,Libellé de la commune,Code du département
7108,21544,Sainte-Colombe-en-Auxois,21


In [54]:
# A comparer avec la 7ème commune
sainte_colombe

,Code postal,Nom de l'élu en 2020,Nom de l'élu en 2014,Libellé de la commune,Noms différents,Tentative de réélection
1556,05135,ROUX,ALMERAS,Sainte-Colombe,True,True
4850,17319,PERUFFO,POURTEAU,Sainte-Colombe,True,True
10957,35262,MALNOE,PETIBON,Sainte-Colombe,True,False
13975,46260,LANDES,LABARTHE,Sainte-Colombe,True,True
22046,69189,DELEIGUE,MASSE,Sainte-Colombe,True,False
24175,76569,BUREL,COLOMBEL,Sainte-Colombe,True,False
5804,21544,FAURE STERNAD,FAURE STERNAD,Sainte-Colombe,False,True
7200,25515,MALFROY,MALFROY,Sainte-Colombe,False,True
10377,33390,THIBEAU,THIBEAU,Sainte-Colombe,False,True
12496,40252,DUTOYA,DUTOYA,Sainte-Colombe,False,True
